In [26]:
from __future__ import division, print_function

from keras.layers import Dot, Dense, Reshape, Embedding, Input
from keras.models import Model
from keras.models import Sequential

## 語彙sizeと、embedding size決定

In [27]:
vocab_size = 5000
embed_size = 300

In [38]:
x0 = Input(shape=(1,))
y0 = Embedding(vocab_size, embed_size,
               embeddings_initializer='glorot_uniform')(x0)
y0 = Reshape((embed_size,))(y0)
word_embedder = Model(x0, y0)
# 文脈語ID --> 文脈語数値ベクトル表現
x1 = Input(shape=(1,))
y1 = Embedding(vocab_size, embed_size,
               embeddings_initializer='glorot_uniform')(x1)
y1 = Reshape((embed_size,))(y1)
context_embedder = Model(x1, y1)
# 内積 --> ロジスティック回帰
y = Dot(axes=-1)([y0, y1])
y = Dense(1, kernel_initializer='glorot_uniform', activation='sigmoid')(y)
model = Model(inputs=[x0, x1], outputs=y)
model.compile(loss='mean_squared_error', optimizer='adam')

http://cookie-box.hatenablog.com/entry/2018/10/14/184801

In [39]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_38 (Embedding)        (None, 1, 300)       1500000     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_39 (Embedding)        (None, 1, 300)       1500000     input_6[0][0]                    
____________________________________________________________________________________________

In [40]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import skipgrams

In [41]:
text = "I love green eggs and ham ."

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [42]:
word2id = tokenizer.word_index
id2word = {v: k for k, v in word2id.items()}

In [43]:
wids = [word2id[w] for w in text_to_word_sequence(text)]
pairs, labels = skipgrams(wids, len(word2id), window_size=1)
print(len(pairs), len(labels))
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
        id2word[pairs[i][0]], pairs[i][0], 
        id2word[pairs[i][1]], pairs[i][1],
        labels[i]))

20 20
(eggs (4), eggs (4)) -> 0
(green (3), love (2)) -> 1
(and (5), eggs (4)) -> 0
(love (2), green (3)) -> 1
(i (1), i (1)) -> 0
(eggs (4), green (3)) -> 1
(eggs (4), and (5)) -> 0
(love (2), i (1)) -> 1
(green (3), eggs (4)) -> 1
(love (2), love (2)) -> 0
